In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import spacy
import re
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
df_fake = pd.read_csv('fake detection.csv')
df_fake.head(2)

,title,text,subject,date,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 162,Unnamed: 163,Unnamed: 164,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168,Unnamed: 169,Unnamed: 170,Unnamed: 171
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df_true = pd.read_csv('true detection.csv')
df_true.head(2)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"


In [4]:
print(df_fake.shape)
print(df_true.shape)

(23502, 172)
(21417, 4)


In [5]:
data=pd.concat([df_fake, df_true],axis=0)

In [6]:
data['class']='Nan'

In [7]:
data

,title,text,subject,date,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 163,Unnamed: 164,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168,Unnamed: 169,Unnamed: 170,Unnamed: 171,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nan
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nan
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nan
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nan
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nan
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nan
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nan
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nan


In [8]:
data['class'][21418:]='True'
data['class'][0:21418]='Fake'

In [9]:
data.shape

(44919, 173)

In [10]:
data.describe()

,title,text,subject,date,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 163,Unnamed: 164,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168,Unnamed: 169,Unnamed: 170,Unnamed: 171,class
count,44919,44919,44898,44898,21,20,20,20,18,18,...,2,2,2,2,2,2,2,2,2,44919
unique,38740,38657,19,2408,11,10,11,10,9,9,...,1,1,1,1,1,1,1,2,1,2
top,Factbox: Trump fills top jobs for his administ...,,politicsNews,"December 20, 2017",free college tuition for illegal aliens,lowered educational standards to accommodate ...,and voting rights for criminals. In MALDEF s ...,supporters of making English the official lan...,while advocates of sanctions against employe...,Suozzi,...,and it s certain that the powers that be will...,it s important to separate the fallout from t...,as those in mainstream media such as The New ...,The Washington Post and the self-anointed fa...,seem to have propped up this pizzeria conspir...,but for the moment,it seems to have collapsed under the weight o...,US_News,"December 28, 2016",True
freq,14,627,11272,182,2,2,2,2,2,2,...,2,2,2,2,2,2,2,1,2,23501


In [11]:
data.subject.unique()

array(['News', 'politics', nan,
       ' of which Soros is a major financier.Mercy Corps: Vis a vis the Arab-Israeli conflict',
       ' high taxes',
       ' Politics According to the Bible and (with Barry Asmus) The Poverty of Nations: A Sustainable Solution.Via: Townhall"',
       'Government News', 'left-news',
       ' claimed that hundreds of alternative media websites were producing  fake news  and  conspiracy  stories and therefore were unreliable as information sources. It wasn t long before the establishment began referencing these politicized lists',
       'US_News', ' and so is Mr. Katzenbach',
       ' fell 5.6 percent Monday. Wynn Resortsslipped 1.2 percent. Las Vegas Sands fell as much as 2.1 percent before closing higher.   LIVE DRILL    Las Vegas has been at the forefront of active shooter training. ( Image Source: sinclairstoryline)Las Vegas Active Shooter Drills Back in 2014',
       ' state systems with outsized pretensions to power have reacted to their environmen

In [12]:
data.dtypes

title           object
text            object
subject         object
date            object
Unnamed: 4      object
                 ...  
Unnamed: 168    object
Unnamed: 169    object
Unnamed: 170    object
Unnamed: 171    object
class           object
Length: 173, dtype: object

In [13]:
data.duplicated().sum()

219

In [14]:
data.drop_duplicates(inplace=True)

In [15]:
data.isnull().sum()

title               0
text                0
subject            11
date               11
Unnamed: 4      44679
                ...  
Unnamed: 168    44698
Unnamed: 169    44698
Unnamed: 170    44698
Unnamed: 171    44698
class               0
Length: 173, dtype: int64

In [16]:
def clean_text(text):
    if isinstance(text, str):  
        clean_text = re.sub(r'[^\w\s]', '', text)
        clean_text = re.sub(r'\d+', '', clean_text)
        return clean_text.lower()  
    else:
        return '' 
data['title'] = data['title'].apply(clean_text)
data['text'] = data['text'].apply(clean_text)

In [17]:
data

,title,text,subject,date,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 163,Unnamed: 164,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168,Unnamed: 169,Unnamed: 170,Unnamed: 171,class
0,donald trump sends out embarrassing new years...,donald trump just couldn t wish all americans ...,News,"December 31, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fake
1,drunk bragging trump staffer started russian ...,house intelligence committee chairman devin nu...,News,"December 31, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fake
2,sheriff david clarke becomes an internet joke...,on friday it was revealed that former milwauke...,News,"December 30, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fake
3,trump is so obsessed he even has obamas name ...,on christmas day donald trump announced that h...,News,"December 29, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fake
4,pope francis just called out donald trump dur...,pope francis used his annual christmas day mes...,News,"December 25, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fake
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21412,fully committed nato backs new us approach on ...,brussels reuters nato allies on tuesday welco...,worldnews,"August 22, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
21413,lexisnexis withdrew two products from chinese ...,london reuters lexisnexis a provider of legal...,worldnews,"August 22, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
21414,minsk cultural hub becomes haven from authorities,minsk reuters in the shadow of disused soviet...,worldnews,"August 22, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
21415,vatican upbeat on possibility of pope francis ...,moscow reuters vatican secretary of state car...,worldnews,"August 22, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [18]:
data['Text_News']=data['text']
data['Title_News']=data['title']

In [19]:
data

,title,text,subject,date,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168,Unnamed: 169,Unnamed: 170,Unnamed: 171,class,Text_News,Title_News
0,donald trump sends out embarrassing new years...,donald trump just couldn t wish all americans ...,News,"December 31, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fake,donald trump just couldn t wish all americans ...,donald trump sends out embarrassing new years...
1,drunk bragging trump staffer started russian ...,house intelligence committee chairman devin nu...,News,"December 31, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fake,house intelligence committee chairman devin nu...,drunk bragging trump staffer started russian ...
2,sheriff david clarke becomes an internet joke...,on friday it was revealed that former milwauke...,News,"December 30, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fake,on friday it was revealed that former milwauke...,sheriff david clarke becomes an internet joke...
3,trump is so obsessed he even has obamas name ...,on christmas day donald trump announced that h...,News,"December 29, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fake,on christmas day donald trump announced that h...,trump is so obsessed he even has obamas name ...
4,pope francis just called out donald trump dur...,pope francis used his annual christmas day mes...,News,"December 25, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fake,pope francis used his annual christmas day mes...,pope francis just called out donald trump dur...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21412,fully committed nato backs new us approach on ...,brussels reuters nato allies on tuesday welco...,worldnews,"August 22, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,brussels reuters nato allies on tuesday welco...,fully committed nato backs new us approach on ...
21413,lexisnexis withdrew two products from chinese ...,london reuters lexisnexis a provider of legal...,worldnews,"August 22, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,london reuters lexisnexis a provider of legal...,lexisnexis withdrew two products from chinese ...
21414,minsk cultural hub becomes haven from authorities,minsk reuters in the shadow of disused soviet...,worldnews,"August 22, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,minsk reuters in the shadow of disused soviet...,minsk cultural hub becomes haven from authorities
21415,vatican upbeat on possibility of pope francis ...,moscow reuters vatican secretary of state car...,worldnews,"August 22, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,moscow reuters vatican secretary of state car...,vatican upbeat on possibility of pope francis ...


In [23]:
from nltk.tokenize import word_tokenize
data['title']=data['title'].apply(lambda x:word_tokenize(str(x)))
data['text']=data['text'].apply(lambda x:word_tokenize(str(x)))

In [25]:
data

,title,text,subject,date,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168,Unnamed: 169,Unnamed: 170,Unnamed: 171,class,Text_News,Title_News
0,"[[, 'donald, ', ,, 'trump, ', ,, 'sends, ', ,,...","[[, 'donald, ', ,, 'trump, ', ,, 'just, ', ,, ...",News,"December 31, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fake,donald trump just couldn t wish all americans ...,donald trump sends out embarrassing new years...
1,"[[, 'drunk, ', ,, 'bragging, ', ,, 'trump, ', ...","[[, 'house, ', ,, 'intelligence, ', ,, 'commit...",News,"December 31, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fake,house intelligence committee chairman devin nu...,drunk bragging trump staffer started russian ...
2,"[[, 'sheriff, ', ,, 'david, ', ,, 'clarke, ', ...","[[, 'on, ', ,, 'friday, ', ,, 'it, ', ,, 'was,...",News,"December 30, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fake,on friday it was revealed that former milwauke...,sheriff david clarke becomes an internet joke...
3,"[[, 'trump, ', ,, 'is, ', ,, 'so, ', ,, 'obses...","[[, 'on, ', ,, 'christmas, ', ,, 'day, ', ,, '...",News,"December 29, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fake,on christmas day donald trump announced that h...,trump is so obsessed he even has obamas name ...
4,"[[, 'pope, ', ,, 'francis, ', ,, 'just, ', ,, ...","[[, 'pope, ', ,, 'francis, ', ,, 'used, ', ,, ...",News,"December 25, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fake,pope francis used his annual christmas day mes...,pope francis just called out donald trump dur...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21412,"[[, 'fully, ', ,, 'committed, ', ,, 'nato, ', ...","[[, 'brussels, ', ,, 'reuters, ', ,, 'nato, ',...",worldnews,"August 22, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,brussels reuters nato allies on tuesday welco...,fully committed nato backs new us approach on ...
21413,"[[, 'lexisnexis, ', ,, 'withdrew, ', ,, 'two, ...","[[, 'london, ', ,, 'reuters, ', ,, 'lexisnexis...",worldnews,"August 22, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,london reuters lexisnexis a provider of legal...,lexisnexis withdrew two products from chinese ...
21414,"[[, 'minsk, ', ,, 'cultural, ', ,, 'hub, ', ,,...","[[, 'minsk, ', ,, 'reuters, ', ,, 'in, ', ,, '...",worldnews,"August 22, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,minsk reuters in the shadow of disused soviet...,minsk cultural hub becomes haven from authorities
21415,"[[, 'vatican, ', ,, 'upbeat, ', ,, 'on, ', ,, ...","[[, 'moscow, ', ,, 'reuters, ', ,, 'vatican, '...",worldnews,"August 22, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,moscow reuters vatican secretary of state car...,vatican upbeat on possibility of pope francis ...


In [ ]:
def remove_stopwords(text):
    stop_words=set(stopwords.words('english'))
    tokens=word_tokenize(str(text))
    filtered_tokens=[word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)
columns_to_process=['text','title']
for column in columns_to_process:
    data[column+'_no_stopwords']=data[column].apply(remove_stopwords)

In [ ]:
data

In [ ]:
sid = SentimentIntensityAnalyzer()
def determine_sentiment(row):
    title_sentiment = sid.polarity_scores(str(row['title_no_stopwords']))['compound']
    text_sentiment = sid.polarity_scores(str(row['text_no_stopwords']))['compound']
    combined_sentiment = (title_sentiment + text_sentiment) / 2
    if combined_sentiment >= 0.05:
        return 'Positive'
    elif combined_sentiment <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'
data['sentiment'] = data.apply(determine_sentiment, axis=1)

In [ ]:
data

In [ ]:
xx=data[['Title_News','Text_News','subject','sentiment']]
yy=data['class']

In [ ]:
xx.columns

In [ ]:
xx[['Title_News', 'Text_News', 'subject', 'sentiment']]=xx[['Title_News', 'Text_News', 'subject', 'sentiment']].apply(LabelEncoder().fit_transform)
print(xx)

In [ ]:
yy = yy.replace({'Fake': 0, 'True': 1})
print(yy)

In [ ]:
scaler=MinMaxScaler()
xx=scaler.fit_transform(xx)

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(xx,yy,test_size=0.2, random_state=42)
classifiers=[
    LogisticRegression(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    SVC()
]
for classifier in classifiers:
    classifier_name=classifier.__class__.__name__
    print(f"Training {classifier_name}...")
    classifier.fit(X_train, y_train)
    y_pred=classifier.predict(X_test)
    accuracy=accuracy_score(y_test, y_pred)
    print(f"\n {classifier_name} \n Accuracy: {accuracy:.4f}\n")